In [2]:
from Alpaca_Websocket_AlgoBot import AlpacaWebSocket
import alpaca_trade_api as tradeapi
import threading
import time

In [3]:
api_key = 'PK6DPMCFN81PFZSA5CCZ'
api_secret = 'sS7OAqSHpZVq05mJdeZ2PtsxU7qqZ0fg1t7fF0fq'
symbol = 'SPY'

In [4]:
alpaca_ws = AlpacaWebSocket(symbol=symbol, api_key=api_key, api_secret=api_secret, full_history=True, historic_days=365*5)
API = tradeapi.REST(api_key, api_secret, base_url='https://paper-api.alpaca.markets')

In [5]:
recorded_last_row = None

def trade_loop(exit_event):
    global recorded_last_row

    try:
        while not exit_event.is_set():
            print("Checking Conditions")
            current_last_row = alpaca_ws.bars_data_period.iloc[-1]

            #Checking if it is new data
            if recorded_last_row is None or not recorded_last_row.equals(current_last_row):
                cross_signal = current_last_row['cross_signal']

                #Execute Buy Signal
                if cross_signal == 1:
                    API.submit_order(
                        symbol=symbol,
                        qty=5,
                        side='buy',
                        type='market',
                        time_in_force='gtc'
                    )
                    price = API.get_latest_trade(symbol=symbol).price
                    print(f"Executed Buy Order of {1} {symbol} stock at ${price}")
        
                #Execute Sell Signal
                elif cross_signal == -1:
                    #Checking if we stocks to sell
                    current_quantity = float(API.get_position(symbol=symbol).qty)
            
                    if current_quantity > 0.0:
                        API.submit_order(
                            symbol=symbol,
                            qty=5,
                            side='sell',
                            type='market',
                            time_in_force='gtc'
                        )
                        price = API.get_latest_trade(symbol=symbol).price
                        print(f"Executed Sell Order of {1} {symbol} stock at ${price}")
                    else:
                        print(f"No stocks to sell for {symbol}")
    
                #Update last recorded row
                recorded_last_row = current_last_row.copy()

            print("Completed Checking Conditions")
            time.sleep(10*60)
    except KeyboardInterrupt:
        print("Executed Trading Bot")
        exit_event.set()
        

In [6]:
def align_time():
    current_time = time.localtime()
    current_mins = current_time.tm_min
    
    if current_mins > 30:
        current_mins -= 30
    
    
    current_seconds = current_time.tm_sec + current_mins*60
    remaining_seconds = (30*60) - current_seconds

    return remaining_seconds

In [7]:
print(f'seconds till 30 mins {align_time()}')

seconds till 30 mins 920


In [8]:
#Start Trade Bot
try:
    exit_event = threading.Event()
    trade_thread = threading.Thread(target=trade_loop, args=(exit_event,))
    trade_thread.start()
    alpaca_ws.start_connection()
except KeyboardInterrupt:
    exit_event.set()
    trade_thread.join()
    print("Trading Bot Terminated")

Checking Conditions
Completed Checking Conditions
Opened Connection
Current Data:
                      timestamp    open    high     low   close  volume   
0     2019-01-28 09:00:00+00:00  264.64  264.64  264.41  264.46   17408  \
1     2019-01-28 09:30:00+00:00  264.56  264.80  264.51  264.75    6157   
2     2019-01-28 10:00:00+00:00  264.80  264.80  264.65  264.71    5838   
3     2019-01-28 10:30:00+00:00  264.78  264.80  264.52  264.57   19124   
4     2019-01-28 11:00:00+00:00  264.56  264.82  264.56  264.77   10700   
...                         ...     ...     ...     ...     ...     ...   
39423 2024-01-25 22:30:00+00:00  487.40  487.50  487.40  487.42   20530   
39424 2024-01-25 23:00:00+00:00  487.47  487.49  486.95  486.95  101478   
39425 2024-01-25 23:30:00+00:00  488.03  488.03  486.94  487.06   18126   
39426 2024-01-26 00:00:00+00:00  487.02  487.12  486.84  486.94   12079   
39427 2024-01-26 00:30:00+00:00  487.00  487.14  486.92  486.94   57171   

          sma_10 

In [12]:
trade_thread.is_alive()

True

In [10]:
exit_event.set()

In [11]:
trade_thread.

<Thread(Thread-5 (trade_loop), started 14332)>